# AR insert ligand workflow

In [1]:
import os
import shutil
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole #Needed to show molecules
from rdkit.Chem.Draw.MolDrawing import MolDrawing, DrawingOptions #Only needed if modifying defaults
import nglview as nv
# from openbabel import openbabel
import MDAnalysis as mda
import concurrent.futures
import numpy as np
import sys
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import mdtraj as md



/home/s.emanuele/anaconda3/envs/STOCAZZO-311/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:160: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


Creating all the folders required

In [2]:
def check_location(message, location, is_directory=False):
    from os.path import isdir, isfile
    if is_directory and isdir(location):
        print(f"{message}: {location} Exists")
    elif isfile(location):
        print(f"{message}: {location} Exists")
    else: print(f"{message}: {location} Not Found")


def create_folder(folder_path):
    try:
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    except OSError as e:
        print(f"Error creating folder '{folder_path}': {e}")


def copy_files(source_directory, destination_directory, file_names):
    try:
        for file_name in file_names:
            source_path = os.path.join(source_directory, file_name)
            destination_path = os.path.join(destination_directory, file_name)
            shutil.copy(source_path, destination_path)
            print(f"File '{file_name}' copied successfully.")
    except Exception as e:
        print(f"Error copying files: {e}")



Functions

In [3]:
# SHOW INDEX ON RDKIT IMAGE
# define the atom indices to be mapped from simulation data 
def mol_with_atom_index(mol): 
    for atom in mol.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx())
    return mol

In [4]:
# READ FILE HEADER
def read_first_line(file_path):
    try:
        with open(file_path, 'r') as file:
            first_line = file.readline().strip()
            return first_line
    except Exception as e:
        return f"An error occurred: {e}"

In [5]:
# LAST LINE FOR SOL
def read_last_line(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            last_line = lines[-1].strip()
            return last_line
    except Exception as e:
        return f"An error occurred: {e}"



In [6]:
def read_lines_in_range(file_path, start_line, end_line):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            lines_in_range = lines[start_line - 1:end_line]
            cleaned_lines = [line.strip() for line in lines_in_range]
            return cleaned_lines
    except Exception as e:
        return [f"An error occurred: {e}"]

In [7]:
# REPLACE ITP LINE
def replace_line_match(file_path, search_string, replacement):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()

        with open(file_path, 'w') as file:
            for line in lines:
                if search_string in line:
                    line = line.replace(search_string, replacement)
                file.write(line)
        return "Replacement completed."
    except Exception as e:
        return f"An error occurred: {e}"

In [8]:
# INSERT LIGAND LINES IN TOPOLOGY
def insert_line_at_number(file_path, line_to_insert, line_number):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()

        lines.insert(line_number - 1, line_to_insert + '\n')

        with open(file_path, 'w') as file:
            file.writelines(lines)
        return "Insertion completed."
    except Exception as e:
        return f"An error occurred: {e}"

In [9]:
def divide_replica_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]

In [10]:
def format_chunk(chunk):
    str_chunk = []
    for n in chunk:
        str_chunk.append(str(n))

    format_chunk = ','.join(str_chunk)
    return format_chunk

In [11]:
def set_working_directory(new_directory):
    try:
        os.chdir(new_directory)
        print(f"Working directory changed to: {os.getcwd()}")
    except OSError as e:
        print(f"Error changing working directory: {e}")

In [12]:
def get_box(replica, n_steps=-10000):
    # TODO to be addded in the previous one to use a for loop less
    box_size = np.loadtxt(f'./{replica}/npt.energy2.xvg', comments=['@', '#'], unpack=True)[1]
    ave_box = np.mean(box_size)
    reduced_ave_box = np.mean(box_size[n_steps:])
    return box_size, ave_box, reduced_ave_box


In [13]:
def get_gmx_energy(replica):
    gmx_energy_cmd = f'echo -e "11 12 13 15 17 48 49 0\n" | gmx energy -s ./{replica}/npt2.tpr -f ./{replica}/npt2.edr -o ./{replica}/npt.energy1.xvg'
    gmx_energy2_cmd = f'echo -e "19 22 23 24 25 0\n" | gmx energy -s ./{replica}/npt2.tpr -f ./{replica}/npt2.edr -o ./{replica}/npt.energy2.xvg'
    !{gmx_energy_cmd}
    !{gmx_energy2_cmd}


In [14]:
def get_gmx_energy_test_press(replica):
    gmx_energy_cmd = f'echo -e "11 12 13 15 17 48 49 0\n" | gmx energy -s ./{replica}/test_press.tpr -f ./{replica}/test_press.edr -o ./{replica}/test_press.xvg'
    !{gmx_energy_cmd}


In [15]:
def get_test_press(replica, n_steps=-10000):
    # TODO to be addded in the previous one to use a for loop less
    pressure = np.loadtxt(f'./{replica}/test_press.xvg', comments=['@', '#'], unpack=True)[5]
    ave_pressure = np.mean(pressure)
    reduced_ave_pressure = np.mean(pressure[n_steps:])
    return pressure, ave_pressure, reduced_ave_pressure


In [16]:
def make_box_editconf(replica, box_size):
    round(box_size, 5)
    gmx_editconf_cmd = f'gmx editconf -f ./{replica}/npt2.pbc.gro -o./{replica}/{replica}.new.gro -box {box_size} {box_size} {box_size}'
    !{gmx_editconf_cmd}

In [17]:
def make_boxes_editconf(replica, box_sizes_dict):
    gmx_editconf_cmd = f'gmx editconf -f ./{replica}/npt2.pbc.gro -o ./{replica}/{replica}.new.gro -box {box_sizes_dict[replica]} {box_sizes_dict[replica]} {box_sizes_dict[replica]}'    
    #print(gmx_editconf_cmd)
    !{gmx_editconf_cmd}

Packmol scripts

In [18]:
# WATER PACKMOL
def make_water_packmol_inp(input_file_path, output_file_path, replacements):
    try:
        with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
            for line in input_file:
                for old_str, new_str in replacements.items():
                    line = line.replace(old_str, new_str)
                output_file.write(line)
        print("File parsing and writing completed successfully.")
    except Exception as e:
        print(f"Error parsing or writing file: {e}")


### Names and definitions

In [19]:
number_of_replicas = 20
ligand_name_prefix = 'Masofaniten'
ligand_chain_name = 'MFT'
protein_name_prefix = 'R2_R3_69aa'
working_directory = '/home/s.emanuele/AR/R2_R3_69aa_Masofaniten/'
protein_apo_folder = '/home/s.emanuele/AR/R2_R3_69aa_MD/'
protein_apo_pdb = f'{protein_apo_folder}{protein_name_prefix}.pdb'
smiles = 'CC(C)(C1=CC=C(C=C1)OCC2=NC(=NC=C2)NS(=O)(=O)C)C3=CC(=C(C(=C3)Cl)OCCCl)C#N'


set_working_directory(f'{working_directory}')
# !ln -s /home/s.emanuele/AR/a99SBdisp.ff /home/s.emanuele/AR/R2_R3_69aa_NT1-269/
# !ln -s /home/s.emanuele/AR/a99SBdisp.ff /home/s.emanuele/AR/R2_R3_69aa_NT1-269/equilibrate


# TODO in case of a rerun
run_minimization = True
run_nvt = True
run_npt = True
run_npt2 = True

# gmx_gpu_mpi_path = 'mpirun -np ${OMPI_NP} /usr/bin/singularity run --nv --bind /data --bind ${HOME} ${HOME}/containers/gromacs_gpu.sif gmx_gpu "$@"'
gmx_gpu_mpi_path = 'mpirun -np 4 /usr/bin/singularity run --nv --bind /data --bind ${HOME} ${HOME}/containers/gromacs_gpu.sif gmx_gpu "$@"'

# chunks of 4 ntomp 4 
# or
# chunks of 5 ntomp 3
mpi_threads = 4 # replicas will be divided by this and simulated in chunks
replica_chunks = 4 # How many directories will be run, should correspond to mpi_threads
ntomp = 4

list_replica_dir = list(range(0, number_of_replicas))
chunk_list_replica_dir = list(divide_replica_chunks(list_replica_dir, replica_chunks))

Working directory changed to: /home/s.emanuele/AR/R2_R3_69aa_Masofaniten


### Minimization

In [20]:
set_working_directory(f'{working_directory}/equilibrate/')

Working directory changed to: /home/s.emanuele/AR/R2_R3_69aa_Masofaniten/equilibrate


In [21]:
if run_minimization is True:
    for replica in range(number_of_replicas):
        gmx_em_grompp_cmd = f'gmx grompp -f ../mdps/minim.mdp -c ./{replica}/{replica}.ions.gro -p ./{replica}/protein_ligand.top -o ./{replica}/em.tpr -maxwarn 2'
        !{gmx_em_grompp_cmd}

                       :-) GROMACS - gmx grompp, 2022 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/s.emanuele/AR/R2_R3_69aa_Masofaniten/equilibrate
Command line:
  gmx grompp -f ../mdps/minim.mdp -c ./0/0.ions.gro -p ./0/protein_ligand.top -o ./0/em.tpr -maxwarn 2

Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file ../mdps/minim.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Setting the LD random seed to -127926289

Generated 4464 of the 4465 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 4465 of the 4465 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein'

Excluding 3 bonded neighbours molecule type 'MFT'

Excluding 1 bonded neighbours molecule type 'SOL'

Excluding 1 bonded neighbours molecule type 'NA'

Excluding 1 bonded neighbours molecul

In [22]:
# mpi_threads 4

In [23]:
chunk_list_replica_dir

[[0, 1, 2, 3],
 [4, 5, 6, 7],
 [8, 9, 10, 11],
 [12, 13, 14, 15],
 [16, 17, 18, 19]]

In [24]:
if run_minimization is True:
    for chunk in chunk_list_replica_dir:
        format_dir_list = format_chunk(chunk)
        print('Running em on the following replicas:',format_dir_list)
        gmx_em_mdrun_cmd = f'{gmx_gpu_mpi_path} mdrun -v -ntomp {ntomp} -deffnm em -multidir {{{format_dir_list}}}'
        print(gmx_em_mdrun_cmd)
        log_minimization = !{gmx_em_mdrun_cmd}


Running em on the following replicas: 0,1,2,3
mpirun -np 4 /usr/bin/singularity run --nv --bind /data --bind ${HOME} ${HOME}/containers/gromacs_gpu.sif gmx_gpu "$@" mdrun -v -ntomp 4 -deffnm em -multidir {0,1,2,3}
Running em on the following replicas: 4,5,6,7
mpirun -np 4 /usr/bin/singularity run --nv --bind /data --bind ${HOME} ${HOME}/containers/gromacs_gpu.sif gmx_gpu "$@" mdrun -v -ntomp 4 -deffnm em -multidir {4,5,6,7}
Running em on the following replicas: 8,9,10,11
mpirun -np 4 /usr/bin/singularity run --nv --bind /data --bind ${HOME} ${HOME}/containers/gromacs_gpu.sif gmx_gpu "$@" mdrun -v -ntomp 4 -deffnm em -multidir {8,9,10,11}
Running em on the following replicas: 12,13,14,15
mpirun -np 4 /usr/bin/singularity run --nv --bind /data --bind ${HOME} ${HOME}/containers/gromacs_gpu.sif gmx_gpu "$@" mdrun -v -ntomp 4 -deffnm em -multidir {12,13,14,15}
Running em on the following replicas: 16,17,18,19
mpirun -np 4 /usr/bin/singularity run --nv --bind /data --bind ${HOME} ${HOME}/con

### NVT

In [25]:
if run_nvt is True:
    for replica in range(number_of_replicas):
        gmx_nvt_grompp_cmd = f'gmx grompp -f ../mdps/nvt.mdp -c ./{replica}/em.gro -p ./{replica}/protein_ligand.top -o ./{replica}/nvt.tpr -maxwarn 2'
        !{gmx_nvt_grompp_cmd}

                       :-) GROMACS - gmx grompp, 2022 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/s.emanuele/AR/R2_R3_69aa_Masofaniten/equilibrate
Command line:
  gmx grompp -f ../mdps/nvt.mdp -c ./0/em.gro -p ./0/protein_ligand.top -o ./0/nvt.tpr -maxwarn 2

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Replacing old mdp entry 'nstxtcout' by 'nstxout-compressed'

NOTE 1 [file ../mdps/nvt.mdp]:
  You have set rlist larger than the interaction cut-off, but you also have
  verlet-buffer-tolerance > 0. Will set rlist using verlet-buffer-tolerance.

Setting the LD random seed to 916453358

Generated 4464 of the 4465 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 4465 of the 4465 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MFT'

turning H bonds into co

In [26]:
if run_nvt is True:
    for chunk in chunk_list_replica_dir:
        format_dir_list = format_chunk(chunk)
        print('Running nvt on the following replicas:',format_dir_list)
        gmx_nvt_mdrun_cmd = f'{gmx_gpu_mpi_path} mdrun -v -ntomp {ntomp} -deffnm nvt -pin on -nb gpu -bonded gpu -pme gpu -pmefft gpu -dlb no -pinoffset 0 -pinstride 0 -multidir {{{format_dir_list}}}'
        log_nvt_mdrun = !{gmx_nvt_mdrun_cmd}


Running nvt on the following replicas: 0,1,2,3
Running nvt on the following replicas: 4,5,6,7
Running nvt on the following replicas: 8,9,10,11
Running nvt on the following replicas: 12,13,14,15
Running nvt on the following replicas: 16,17,18,19


In [27]:
if run_nvt is True:
    nvt_logs, performance_log = [], {}
    for replica in range(number_of_replicas):
        log_path = f'./{replica}/nvt.log'
        with open(f"{log_path}", 'r') as fp:
            last_line = len(fp.readlines())
        nvt_logs.extend(read_lines_in_range(log_path, last_line-75, last_line))
        performance_log[replica] = (read_lines_in_range(log_path, last_line-2, last_line-2))
        #for line in nvt_logs:
        #    print(line)
    for k,v in performance_log.items():
        print(f'NVT Replica: {k} {v}')
        

NVT Replica: 0 ['Performance:      100.144        0.240']
NVT Replica: 1 ['Performance:       98.560        0.244']
NVT Replica: 2 ['Performance:      105.260        0.228']
NVT Replica: 3 ['Performance:      107.106        0.224']
NVT Replica: 4 ['Performance:      100.213        0.239']
NVT Replica: 5 ['Performance:      101.132        0.237']
NVT Replica: 6 ['Performance:      108.130        0.222']
NVT Replica: 7 ['Performance:      107.024        0.224']
NVT Replica: 8 ['Performance:       99.284        0.242']
NVT Replica: 9 ['Performance:      102.487        0.234']
NVT Replica: 10 ['Performance:      106.458        0.225']
NVT Replica: 11 ['Performance:      108.603        0.221']
NVT Replica: 12 ['Performance:      101.495        0.236']
NVT Replica: 13 ['Performance:      103.096        0.233']
NVT Replica: 14 ['Performance:      102.809        0.233']
NVT Replica: 15 ['Performance:      105.852        0.227']
NVT Replica: 16 ['Performance:       96.657        0.248']
NVT Rep

### NPT

In [28]:
if run_npt is True:
    for replica in range(number_of_replicas):
        gmx_nvt_grompp_cmd = f'gmx grompp -f ../mdps/npt.mdp -c ./{replica}/nvt.gro -p ./{replica}/protein_ligand.top -o ./{replica}/npt.tpr -maxwarn 2'
        !{gmx_nvt_grompp_cmd}

                       :-) GROMACS - gmx grompp, 2022 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/s.emanuele/AR/R2_R3_69aa_Masofaniten/equilibrate
Command line:
  gmx grompp -f ../mdps/npt.mdp -c ./0/nvt.gro -p ./0/protein_ligand.top -o ./0/npt.tpr -maxwarn 2

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Replacing old mdp entry 'nstxtcout' by 'nstxout-compressed'
Setting the LD random seed to -273712577

Generated 4464 of the 4465 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 4465 of the 4465 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MFT'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into constraints

In [29]:
if run_npt is True:
    for chunk in chunk_list_replica_dir:
        format_dir_list = format_chunk(chunk)
        print('Running npt on the following replicas:',format_dir_list)
        gmx_npt_mdrun_cmd = f'{gmx_gpu_mpi_path} mdrun -v -ntomp {ntomp} -deffnm npt -pin on -nb gpu -bonded gpu -pme gpu -pmefft gpu -dlb no -pinoffset 0 -pinstride 0 -multidir {{{format_dir_list}}}'
        log_npt_mdrun = !{gmx_npt_mdrun_cmd}

Running npt on the following replicas: 0,1,2,3
Running npt on the following replicas: 4,5,6,7
Running npt on the following replicas: 8,9,10,11
Running npt on the following replicas: 12,13,14,15
Running npt on the following replicas: 16,17,18,19


In [30]:
if run_npt is True:
    npt_logs, performance_log = [], {}
    for replica in range(number_of_replicas):
        log_path = f'./{replica}/npt.log'
        with open(f"{log_path}", 'r') as fp:
            last_line = len(fp.readlines())
        npt_logs.extend(read_lines_in_range(log_path, last_line-75, last_line))
        performance_log[replica] = (read_lines_in_range(log_path, last_line-2, last_line-2))
        #for line in npt_logs:
        #    print(line)
    for k,v in performance_log.items():
        print(f'NVT Replica: {k} {v}')
        

NVT Replica: 0 ['Performance:      183.979        0.130']
NVT Replica: 1 ['Performance:      190.888        0.126']
NVT Replica: 2 ['Performance:      192.229        0.125']
NVT Replica: 3 ['Performance:      193.655        0.124']
NVT Replica: 4 ['Performance:      184.270        0.130']
NVT Replica: 5 ['Performance:      190.250        0.126']
NVT Replica: 6 ['Performance:      194.859        0.123']
NVT Replica: 7 ['Performance:      194.507        0.123']
NVT Replica: 8 ['Performance:      179.315        0.134']
NVT Replica: 9 ['Performance:      187.216        0.128']
NVT Replica: 10 ['Performance:      193.430        0.124']
NVT Replica: 11 ['Performance:      187.295        0.128']
NVT Replica: 12 ['Performance:      179.012        0.134']
NVT Replica: 13 ['Performance:      193.839        0.124']
NVT Replica: 14 ['Performance:      191.257        0.125']
NVT Replica: 15 ['Performance:      194.098        0.124']
NVT Replica: 16 ['Performance:      185.557        0.129']
NVT Rep

### NPT2

In [31]:
if run_npt2 is True:
    for replica in range(number_of_replicas):
        gmx_nvt_grompp_cmd = f'gmx grompp -f ../mdps/npt2.mdp -c ./{replica}/npt.gro -p ./{replica}/protein_ligand.top -o ./{replica}/npt2.tpr -maxwarn 2'
        !{gmx_nvt_grompp_cmd}

                       :-) GROMACS - gmx grompp, 2022 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/s.emanuele/AR/R2_R3_69aa_Masofaniten/equilibrate
Command line:
  gmx grompp -f ../mdps/npt2.mdp -c ./0/npt.gro -p ./0/protein_ligand.top -o ./0/npt2.tpr -maxwarn 2

Ignoring obsolete mdp entry 'title'
Ignoring obsolete mdp entry 'ns_type'
Replacing old mdp entry 'nstxtcout' by 'nstxout-compressed'
Setting the LD random seed to -134054

Generated 4464 of the 4465 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 4465 of the 4465 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein'

turning H bonds into constraints...

Excluding 3 bonded neighbours molecule type 'MFT'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into constraints.

In [32]:
# chunk_list_replica_dir = [[12, 13, 14, 15],[16, 17, 18, 19]]

In [33]:
if run_npt2 is True:
    for chunk in chunk_list_replica_dir:
        format_dir_list = format_chunk(chunk)
        print('Running npt2 on the following replicas:',format_dir_list)
        gmx_npt2_mdrun_cmd = f'{gmx_gpu_mpi_path} mdrun -v -ntomp {ntomp} -deffnm npt2 -pin on -nb gpu -bonded gpu -pme gpu -pmefft gpu -dlb no -pinoffset 0 -pinstride 0 -multidir {{{format_dir_list}}}'
        print(gmx_npt2_mdrun_cmd)
        !{gmx_npt2_mdrun_cmd}

Running npt2 on the following replicas: 0,1,2,3
mpirun -np 4 /usr/bin/singularity run --nv --bind /data --bind ${HOME} ${HOME}/containers/gromacs_gpu.sif gmx_gpu "$@" mdrun -v -ntomp 4 -deffnm npt2 -pin on -nb gpu -bonded gpu -pme gpu -pmefft gpu -dlb no -pinoffset 0 -pinstride 0 -multidir {0,1,2,3}
 :-) GROMACS - gmx mdrun, 2022.3-plumed_2.8.3-dev-20220902-47c9856ee2-dirty-unknown (-:

Executable:   /usr/local/gromacs/bin/gmx_gpu
Data prefix:  /usr/local/gromacs
Working dir:  /home/s.emanuele/AR/R2_R3_69aa_Masofaniten/equilibrate
Command line:
  gmx_gpu mdrun -v -ntomp 4 -deffnm npt2 -pin on -nb gpu -bonded gpu -pme gpu -pmefft gpu -dlb no -pinoffset 0 -pinstride 0 -multidir 0 1 2 3

Reading file npt2.tpr, VERSION 2022 (single precision)
Reading file npt2.tpr, VERSION 2022 (single precision)
Reading file npt2.tpr, VERSION 2022 (single precision)
Reading file npt2.tpr, VERSION 2022 (single precision)
Changing nstlist from 10 to 80, rlist from 1.005 to 1.163

Changing nstlist from 10 to

In [34]:
if run_npt2 is True:
    npt2_logs, performance_log = [], {}
    for replica in range(number_of_replicas):
        log_path = f'./{replica}/npt2.log'
        with open(f"{log_path}", 'r') as fp:
            last_line = len(fp.readlines())
        npt2_logs.extend(read_lines_in_range(log_path, last_line-75, last_line))
        performance_log[replica] = (read_lines_in_range(log_path, last_line-2, last_line-2))
        #for line in npt2_logs:
        #    print(line)
    for k,v in performance_log.items():
        print(f'NVT Replica: {k} {v}')
        

NVT Replica: 0 ['Performance:      184.085        0.130']
NVT Replica: 1 ['Performance:      186.882        0.128']
NVT Replica: 2 ['Performance:      191.789        0.125']
NVT Replica: 3 ['Performance:      189.854        0.126']
NVT Replica: 4 ['Performance:      177.378        0.135']
NVT Replica: 5 ['Performance:      182.766        0.131']
NVT Replica: 6 ['Performance:      185.174        0.130']
NVT Replica: 7 ['Performance:      189.657        0.127']
NVT Replica: 8 ['Performance:      183.363        0.131']
NVT Replica: 9 ['Performance:      188.104        0.128']
NVT Replica: 10 ['Performance:      187.981        0.128']
NVT Replica: 11 ['Performance:      186.480        0.129']
NVT Replica: 12 ['Performance:      177.791        0.135']
NVT Replica: 13 ['Performance:      183.815        0.131']
NVT Replica: 14 ['Performance:      180.293        0.133']
NVT Replica: 15 ['Performance:      173.736        0.138']
NVT Replica: 16 ['Performance:      179.867        0.133']
NVT Rep